# Lesson 3: Reflection and Blogpost Writing

## Setup

In [1]:
!python -m pip install ollama
!python -m pip install openai
## !python -m pip install openllama
## !python -m pip install getpass

  Using cached openai-1.35.1-py3-none-any.whl.metadata (21 kB)
  Using cached distro-1.9.0-py3-none-any.whl.metadata (6.8 kB)
  Using cached pydantic-2.7.4-py3-none-any.whl.metadata (109 kB)
  Using cached tqdm-4.66.4-py3-none-any.whl.metadata (57 kB)
  Using cached annotated_types-0.7.0-py3-none-any.whl.metadata (15 kB)
  Using cached pydantic_core-2.18.4-cp312-cp312-macosx_11_0_arm64.whl.metadata (6.5 kB)
Using cached openai-1.35.1-py3-none-any.whl (326 kB)
Using cached distro-1.9.0-py3-none-any.whl (20 kB)
Using cached pydantic-2.7.4-py3-none-any.whl (409 kB)
Using cached pydantic_core-2.18.4-cp312-cp312-macosx_11_0_arm64.whl (1.8 MB)
Using cached tqdm-4.66.4-py3-none-any.whl (78 kB)
Using cached annotated_types-0.7.0-py3-none-any.whl (13 kB)


In [5]:
import os 
import getpass
from openai import OpenAI

local = False
if local:
    llm_config = None
else:
    llm_config = {"model": "gpt-4o"}
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ") 

## The task!

In [3]:
task = '''
       Discuss a news item to help a Human Journalist write an engaging news article about a subject given by the User. 
       After discussing propose an article (including a headline).
       Make sure the article is within 100 words. 
       Every article should have a headline, a lead, and a body.
       Het artikel moet in het Nederlands zijn.
       '''


## Create 2 Journallist agents

In [4]:
from autogen import ConversableAgent

seriousJournalist = ConversableAgent(
    name="Serious Journalist",
    is_termination_msg=lambda x: x.get("content", "").find("TERMINATE") >= 0,
    llm_config=llm_config,
    system_message="""You are a Serious Journalist. You discuss a news item 
                with another Agent, a Sensational Journalist, and you want 
                to make sure all facts are checked before an article is published. 
                Give feedback when it is possible to improve the quality of the content, 
                or when facts have to be checked.
                De conversatie moet in het Nederlands zijn. 
                Als de Sensational Journalist wat snel conclusies wil trekken, 
                moet je hem/haar corrigeren. Laat gerust merken dat je het irritant 
                vindt als iemand te snel concllusies wil trekken.""",
    human_input_mode="ALWAYS"
)

sensationalJournalist = ConversableAgent(
    name="Sensational Journalist",
    system_message="""You are a Journalist. You write engaging news articles (with title) 
        on given topics. You like to make it Sensational, to make it appealing for the reader! 
        But you have to discuss 
        with another Agent, a Serious Journalist, who wants to be sure 
        all facts are checked. """,
    llm_config=llm_config,
    human_input_mode="NEVER",
)

ModuleNotFoundError: No module named 'autogen'

In [20]:
reply = sensationalJournalist.generate_reply(messages=[{"content": task, "role": "user"}])
print(reply)

## Adding reflection 

Create a critic agent to reflect on the work of the writer agent.

In [22]:
res = seriousJournalist.initiate_chat(
    recipient=sensationalJournalist,
    message=task,
    max_turns=2,
    summary_method="last_msg"
)

Serious Journalist (to Sensational Journalist):


       Discuss a news item to help a Human Journalist write an engaging news article about a subject given by the User. 
       After discussing propose an article (including a headline).
       Make sure the article is within 100 words. 
       Every article should have a headline, a lead, and a body.
       Het artikel moet in het Nederlands zijn.
       

--------------------------------------------------------------------------------
Sensational Journalist (to Serious Journalist):

Prima! Wat is het onderwerp waar je graag een nieuwsartikel over zou willen schrijven?

--------------------------------------------------------------------------------
Serious Journalist (to Sensational Journalist):

hpohpoho, niet te snel!

--------------------------------------------------------------------------------
Sensational Journalist (to Serious Journalist):

Sorry, ik ben hier om je te helpen! Wat zou je willen dat het onderwerp van ons nieuws

## Nested chat

In [ ]:
facts_checker = ConversableAgent(
    name="Facts Checker",
    llm_config=llm_config,
    system_message="""You are a reviewer, known for 
        your ability to check facts. 
        When you see a fact that could be incorrect, 
        you should point it out. 
        Then you think of a way to check if it is a fact. 
        If you can do it yourself: do it (and tell it), 
        otherwise tell the Journalists how to check it.""",
)

legal_reviewer = ConversableAgent(
    name="Legal Reviewer",
    llm_config=llm_config,
    system_message="You are a legal reviewer, known for "
        "your ability to ensure that content is legally compliant "
        "and free from any potential legal issues. "
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role.",
)

ethics_reviewer = ConversableAgent(
    name="Ethics Reviewer",
    llm_config=llm_config,
    system_message="You are an ethics reviewer, known for "
        "your ability to ensure that content is ethically sound "
        "and free from any potential ethical issues. " 
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role. ",
)

meta_reviewer = ConversableAgent(
    name="Meta Reviewer",
    llm_config=llm_config,
    system_message="You are a meta reviewer, you aggragate and review "
    "the work of other reviewers and give a final suggestion on the content.",
)

## Orchestrate the nested chats to solve the task

In [ ]:
def reflection_message(recipient, messages, sender, config):
    return f'''Discuss the following news item. 
            \n\n {recipient.chat_messages_for_summary(sender)[-1]['content']}'''

review_chats = [
    {
     "recipient": facts_checker, 
     "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'Checker': '', 'FactCheck': ''}. Here Fact Checker should be your role",},
     "max_turns": 3},
    {
    "recipient": legal_reviewer, "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'Reviewer': '', 'Review': ''}.",},
     "max_turns": 1},
    {"recipient": ethics_reviewer, "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'reviewer': '', 'review': ''}",},
     "max_turns": 1},
     {"recipient": meta_reviewer, 
      "message": "Aggregrate feedback from all reviewers and give final suggestions on the writing.", 
     "max_turns": 1},
]


In [ ]:
seriousJournalist.register_nested_chats(
    review_chats,
    trigger=sensationalJournalist,
)

res = seriousJournalist.initiate_chat(
    recipient=sensationalJournalist,
    message=task,
    max_turns=2,
    summary_method="last_msg"
)

## Get the summary

In [ ]:
print(res.summary)